In [231]:
!pip install -q pyomo

In [232]:
from pyomo.environ import *

1[R]\
$\ max\ 3C_1+5C_2+4C_3+6C_4+6C_5+8C_6+8C_7+6C_8+4C_9+6C_10\\
3C_1+5C_2+4C_3+6C_4+6C_5+8C_6+8C_7+6C_8+4C_9+6C_10\geq26\\
 \sum(Ci)\leq 9,  \sum(Ci)\geq 5\\
 C_5+C_7\leq 1\\C_3+C_9\leq1\\C_7+C_2\leq1\\C_10 -C1\leq0\\C_6+C_1\leq1\\
 C_2+C_4\leq1\\C_8+C_10\leq1
$

#
in order to solve this question i am choosing the allowed courses as **variable which can take value either 0 or 1**. where 0 indicates course is not selected and 1 indicates course is registered for current semester. And **credit of respective course will become coeffiecint of these variables in objective function**.
in this way i made an objective function.
condition was given that minimum credit should be greater than equal to 26. so this become one of the required constraint as shown above.
other constraints which was mentioned in the problem is formulated as shown above.
whenever there is coinciding time table i formulated that only one course one can register in that time slot.

In [233]:
import numpy as np

In [234]:
model_milp = ConcreteModel()

In [235]:
N = 10 
col_indices=np.arange(N)
print(col_indices)
M=10
row_indices=np.arange(M)
print(row_indices)
ub = np.array([-26,-5,9,1,1,1,0,1,1,1])

[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]


In [236]:
#defining coeffiecint of objective function( it is nothing but credits associated with the  courses.)
obj_coef_c = np.array([3,5,4,6,6,8,8,6,4,6]) 

In [237]:
#defining all constraints coeff in array form. constraints are arranged in sequence as shown above in  problem formulation.
constr_coef=np.array([[-3,-5,-4,-6,-6,-8,-8,-6,-4,-6],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,],[1,1,1,1,1,1,1,1,1,1],
                      [0,0,0,0,1,0,1,0,0,0],[0,0,1,0,0,0,0,0,1,0],[0,1,0,0,0,0,1,0,0,0],
                      [-1,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0], [0,1,0,1,0,0,0,0,0,0], [0,0,0,0,0,0,0,1,0,1]])


In [238]:
#i am solving this problem by assuming variables can take binary value only i.e 0 or 1)
model_milp.C =Var(col_indices,domain=Binary)


In [239]:
model_milp.constraint=ConstraintList()

In [240]:
for i in row_indices:
   model_milp.constraint.add(sum(constr_coef[i,j]*model_milp.C[j] for j in col_indices)<=ub[i])
   


In [ ]:
model_milp.objective = Objective(expr =summation(obj_coef_c,model_milp.C) , sense=maximize)

model_milp.pprint() 

In [242]:
!apt-get install -y -qq coinor-cbc

In [243]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(model_milp)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

In [ ]:
# display solution
print('\nObjective = ', model_milp.objective())

print('\nDecision Variables')
for i in col_indices:
  print('C[',i,'] = ', model_milp.C[i].value)
print('\nConstraints')
model_milp.constraint.display()

3[R] 
optimal value of objective function is 35.\
Decision Variables\
C[ 1 ] =  1.0\
C[ 2 ] =  0.0\
C[ 3 ] =  1.0\
C[ 4 ] =  1.0\
C[ 5 ] =  0.0\
C[ 6 ] =  1.0\
C[ 7 ] =  1.0\
C[ 8 ] =  0.0\
C[ 9 ] =  0.0\
C[ 10 ] =  1.0\
those variables which is  taking value equal to 1 should be choosen in this sem for maximum credit.


**4.[R]** 
**solving above problem using Real variables**.

In [246]:
model_real=ConcreteModel()


In [247]:
model_real.C=Var(col_indices,domain=Reals)

In [248]:
lb_real=np.array([0,0,0,0,0,0,0,0,0,0])
ub_real=np.array([1,1,1,1,1,1,1,1,1,1])

In [249]:
model_real.constraint=ConstraintList()

In [250]:
for i in row_indices:
   model_real.constraint.add(sum(constr_coef[i,j]*model_real.C[j] for j in col_indices)<=ub[i])
   

In [251]:
model_real.objective = Objective(expr =summation(obj_coef_c,model_real.C) , sense=maximize)

In [252]:
for i in col_indices:
   model_real.C[i].setlb(lb_real[i])
   model_real.C[i].setub(ub_real[i])

In [ ]:
model_real.pprint() 

In [ ]:
result = opt_cbc.solve(model_real)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

In [ ]:
# display solution
print('\nObjective = ', model_real.objective())

print('\nDecision Variables')
for i in col_indices:
  print('C[',i,'] = ', model_real.C[i].value)
print('\nConstraints')
model_milp.constraint.display()

#5[R]
yes the solution of MILP can be obtained by merely rounding off the LP ,
what i observed till now is that ,at optimal condition value of objective function
and variable are more or less same. In the previous problem infact i got 
same value of objective function and variablees for both real and integer case. 

6[R]\
Here we need to add two new constraint in order to add a new course C11
C11+C8<=1  and  C11+C7<=1 
objective function will also get change with addition of this new course \
required modification is to add 9*C11 in existing objective function . 


In [256]:
# adding a new variable C11  
N=11
model_milp.C11=Var(domain=Binary)

In [257]:
#adding two  new constraint( C11+C8<=1)#note that  C[7] is C8 and C[6] is C7
model_milp.constraint.add((model_milp.C11+model_milp.C[7])<=1)
model_milp.constraint.add((model_milp.C11+model_milp.C[6])<=1)

In [258]:
model_milp.new_objective=Objective(expr=9*model_milp.C11+summation(obj_coef_c,model_milp.C),sense=maximize)


In [259]:
model_milp.objective.deactivate()

In [ ]:
model_milp.pprint()

In [ ]:
 result = opt_cbc.solve(model_milp)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

**7[R]**\
**optimal value of the objective function and variuables are printed below**.

In [ ]:
# display solution
print('\nObjective = ', model_milp.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('C[',i,'] = ', model_milp.C[i].value)
print('C[11] = ',model_milp.C11.value)
print('\nConstraints')
model_milp.constraint.display()

8[R]

In [263]:
model_milp.C.domain=Reals
for i in col_indices:
  model_milp.C[i].setlb(lb_real[i])
  model_milp.C[i].setub(ub_real[i])
model_milp.C11.domain=Reals

In [ ]:
model_milp.pprint()

In [ ]:
 result = opt_cbc.solve(model_milp)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

In [ ]:
# display solution
print('\nObjective = ', model_milp.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('C[',i,'] = ', model_milp.C[i].value)
print('C[11] = ',model_milp.C11.value)
print('\nConstraints')
model_milp.constraint.display()

#8[R]  when i have changed variable constraint from integer to real ,
 i got value of objective function =**41.999999999992** when variable is considered real.\
 whereas when variable is integer value of objective function =**42**
 this difference arises because in real variable case at optimal condition the value of **variable C[6] which should be zero** but because floating point error this value **came close to zero but not zero**.\
 **so if we are going to roundoff we will get same ans which we obtained in integer case**. 